In [6]:
import pandas as pd
%pylab inline
import seaborn as sns
import regex as re
import plotly.express as px
from IPython.display import Image

Populating the interactive namespace from numpy and matplotlib


/Users/JB/opt/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['re']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Read the file & its the tabs

We have 3 importants sheets for this exercise:

- `All practices`
- `Target practice postocde`
- `Usage by practice`

Each of these tab will be read into dataframe

- df_pract for `All practice`
- df_targ for `Target practice postcodes`
- df_usge for `Usage by practice`

Let' go!



In [2]:
# read all practices tab
df_pract=pd.read_excel('Customer Success Analyst task.xlsx', sheet_name='All practices')
# let's have a look now
df_pract.head(3)

,Address,accuRx_id
0,"Manor Surgery, Osler Road, Oxford, OX3 9BP",1
1,"Hollow Way Medical Centre, 18 Ivy Close, Oxfor...",2
2,"114 High Road, London, N15 6JR",3


In [3]:
df_targ=pd.read_excel('Customer Success Analyst task.xlsx', sheet_name='Target practice postcodes')
df_usge=pd.read_excel('Customer Success Analyst task.xlsx', sheet_name='Usage by practice')
df_targ.head(3) # look at the first 3 rows

,Postcode
0,WN3 6PR
1,WS1 1YB
2,WD19 7SF


# TASK STRATEGY

- Retrieve the postcode from address column
- Perform a join `(inner join)` between the All practices paractice and the postode.
- then perform a last another join  linking the previous results to the Usage by practice tab using the accuRX column as a look up reference to the number messages sent last week.<br><br>

Estimated time 5mn. ![title](smiley.jpg)

### Postcode extraction

In [7]:
postecode_pattern= r'([A-Za-z]{1,2}[0-9Rr][0-9A-Za-z]? [0-9][ABD-HJLNP-UW-Zabd-hjlnp-uw-z]{2})'
extract_column= df_pract.columns.get_loc('Address')

In [8]:
df_pract['Postcode']=df_pract['Address'].str.extract(postecode_pattern)

In [9]:
##Inner Join
answer= df_pract.merge(df_targ, on='Postcode', how= 'inner').merge(df_usge, on='accuRx_id', how='left')
df=answer.copy()

In [22]:
print('The number of messages sent last week y the 20 target practices is: ',answer['Messages sent last week'].sum())

The number of messages sent last week y the 20 target practices is:  9736.0


# Done   ![title](done.png)  <br>  Look at your answer below. 


In [10]:
# Here is your answer
answer

,Address,accuRx_id,Postcode,Messages sent last week
0,"Fulford Grange, Micklefield Lane, Rawdon, Leed...",28,LS19 6BA,NaN
1,"Fulford Grange, Micklefield Lane, Rawdon, Leed...",32,LS19 6BA,NaN
2,"Hazeley Road, Twyford, Winchester, SO21 1QY",90,SO21 1QY,786.0
3,"Bevan House, 17 Beecham Court, Smithy Brook Ro...",94,WN3 6PR,44.0
4,"Winchester, SO21 3LE",128,SO21 3LE,1168.0
5,"Oxhey Drive, Watford, WD19 7SF",138,WD19 7SF,194.0
6,"2 Winchester Mews, NW3 3NP",139,NW3 3NP,991.0
7,"Cogges Surgery, 12 Cogges Hill Road, Witney, O...",148,OX28 3FS,282.0
8,"6 Cunningham Way, Watford, WD25 7NL",238,WD25 7NL,503.0
9,"Sir John Kirk Close Surgery, 3 Sir John Kirk C...",257,SE5 0BB,NaN


# EXTRA: CLEANING JUST FOR YOU :)

In [11]:
df['address']=df_pract['Address'].str.replace(postecode_pattern,'')# creatin a new adress column
df.drop('Address',1, inplace=True ) # dropping the old address columns
df=df[['accuRx_id','address', 'Postcode', 'Messages sent last week']] # re-ordering the order of the columns

In [12]:
df['address']=df.address.str.strip().str[:-1] # removing the last comma

In [13]:
df['City']=df.address.str.split().str[-1] # getting the city

In [14]:
df['address']=df.address.str.split(',').str[:-1].str.join(',')

In [15]:
# mask
mask=df['address'].str.split(',').str.len().eq(1)
df.loc[mask,'address']='No Name, '+df.loc[mask,'address']
df.loc[12, 'address']='Muswell Hill, 1 Dukes Avenue' # reverse this, you will see why

In [16]:
df['Practice Name']=df.address.str.split(',').str[0]
df['Address']=df.address.str.split(',').str[1]
df.drop('address', 1,inplace=True )

In [17]:
df=df[['accuRx_id','Practice Name','Address','Postcode','Messages sent last week', 'City']]# re-odring columns

In [18]:
df.sort_values(by=['Messages sent last week'], ascending=False, inplace=True) # sorting the message sent column
df.head()# this how your dataframe look like

,accuRx_id,Practice Name,Address,Postcode,Messages sent last week,City
12,311,Muswell Hill,1 Dukes Avenue,HP22 6LD,1501.0,London
13,312,No Name,Gordon Road,HA9 8HD,1295.0,London
4,128,Botley Medical Centre,Elms Road,SO21 3LE,1168.0,Oxford
20,489,Fulford Grange,Micklefield Lane,ME4 3EP,1096.0,Leeds
6,139,Temple Cowley Health Centre,Temple Road,NW3 3NP,991.0,Oxford


In [20]:
df['Messages sent last week'].sum()

9736.0

In [ ]:
# your csv file CLEANED 
#df.to_csv('Your_refined_csv')